# Gradient-based Edge Detection

<table style="margin-left: 0px; margin-top: 20px; margin-bottom: 20px;">
<tr>
<td style="width:120px; padding-top: 10px; padding-bottom: 10px;"><img src="assets/Logo-TSP-IPP.png" style="height: 130px;"></td>
<td style="padding-left: 12px;">
<table style="width: 100%;">
<tr>
<th style="text-align: left; width: 80px;">File</th>
<td style="text-align: left;">OpenCV_GradientEdgeDetection.ipynb</td>
</tr>
<tr>
<th style="text-align: left;">Author</th>
<td style="text-align: left;">Nicolas ROUGON</td>
</tr>
<tr>
<th style="text-align: left;">Affiliation</th>
<td style="text-align: left;">Institut Polytechnique de Paris &nbsp;|&nbsp; Telecom SudParis &nbsp;|&nbsp; ARTEMIS Department</td>
</tr>
<tr>
<th style="text-align: left;">Date</th>
<td style="text-align: left;">July 25, 2022</td>
</tr>
<tr>
<th style="text-align: left;">Description</th>
<td style="text-align: left;">OpenCV sample routine &nbsp;>&nbsp; Gradient-based edge detection</td>
</tr>
</table>
</td>
</tr>
</table>
    
<b>OpenCV Documentation</b>
<ul style="margin-top: 2px;">
<li><a href="https://docs.opencv.org/4.6.0/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1">Gaussian filtering</a></li>
<li><a href="https://docs.opencv.org/4.6.0/dd/d1a/group__imgproc__feature.html#ga04723e007ed888ddf11d9ba04e2232de">Gradient-based edge detection with Canny post-processing</a> <sup>(*)</sup></li>
</ul>

<b>Tutorial</b>
<ul style="margin-top: 1px;">
<li><a href="https://docs.opencv.org/4.6.0/da/d5c/tutorial_canny_detector.html">Canny post-processing</a> <sup>(*)</sup></li>
</ul>
<sup>(*)</sup>&nbsp; non-maximum suppression + hysteresis thresholding


In [1]:
import cv2 as cv
import numpy as np
import sys

sys.path.append("C:\\Users\\sabri\\OneDrive\\Documentos\\TP2 - Edge detection\\Notebooks\\")  # TO BE SET ACCORDING TO YOUR LOCAL INSTALLATION
from OpenCV_Image_Utilities import *          # Provides routine overlay_uchar_image()

##
## Global Variables
##

# - Gaussian smoothing
trackbarStdDeviation_name = "10*StdDev"
trackbarStdDeviation_max_value = 75         # Trackbar max value
trackbarStdDeviation_min_value = 1          # Trackbar min value
trackbarStdDeviation_value = 1              # Trackbar value
trackbarStdDeviation_stepsize = 0.1         # Quantization step
sigma_default_value = 1.0                   # Gaussian kernel std deviation default value

# - Contrast map (Sobel gradient kernel)
SobelKernelSize = 3                         # Sobel gradient kernel size

# - Contrast norm
trackbarContrastNorm_name = "L2 norm"
trackbarContrastNorm_max_value = 1          # Trackbar max value
trackbarContrastNorm_value = 1              # Trackbar value
useL2NormForContrast_default_value = True

# - Upper threshold
trackbarUpperThreshold_name = "Upper"     
trackbarUpperThreshold_max_value = 255      # Trackbar max value
trackbarUpperThreshold_value = 128          # Trackbar value
upperThreshold_default_value = 128.0

# - % of upper threshold, defining lower threshold
trackbarUpperThresholdRatio_name = "% Upper"
trackbarUpperThresholdRatio_max_value = 100  # Trackbar max value
trackbarUpperThresholdRatio_value = 100      # Trackbar value
upperThresholdRatio_default_value = 100.0

# Edge map overlay
edge_color = 255
edge_color3 = (0,0,255)

##
## IMAGE PIPELINE COMPONENTS 
##

#
# Graphical User Interface 
#
def create_GUI():
                                  # Windows
    global window_in_name         # Displays original image
    global window_settings_name   # GUI
    global window_out_name        # Displays edge map
    global window_overlay_name    # Displays edge map onto original image
    
    window_name_prefix = 'OpenCV Demo |  Edge detection > '
    
    # Create windows
    # - for original image
    window_in_name = window_name_prefix + 'Original image'
    cv.namedWindow(window_in_name, cv.WINDOW_AUTOSIZE)
 
    # - for hyperparameter settings
    window_settings_name = window_name_prefix + 'Settings'
    cv.namedWindow(window_settings_name, cv.WINDOW_AUTOSIZE)
    
    # - for edge map
    window_out_name = window_name_prefix + 'Edge map'
    cv.namedWindow(window_out_name, cv.WINDOW_AUTOSIZE)

    # - for edge map overlay
    window_overlay_name = window_name_prefix + "Detected edges"
    cv.namedWindow(window_overlay_name, cv.WINDOW_AUTOSIZE)
    
    # Create trackbars
    # - for Gaussian kernel std deviation
    cv.createTrackbar(trackbarStdDeviation_name, window_settings_name, 0,
                      trackbarStdDeviation_max_value, process_display_callback)
    cv.setTrackbarMin(trackbarStdDeviation_name, window_settings_name, 
                      trackbarStdDeviation_min_value)
    
    # - for contrast norm
    cv.createTrackbar(trackbarContrastNorm_name, window_settings_name, 0,
                      trackbarContrastNorm_max_value, process_display_callback)
    
    # - for upper hysteresis threshold
    cv.createTrackbar(trackbarUpperThreshold_name, window_settings_name, 0,
                      trackbarUpperThreshold_max_value, process_display_callback)
    
    # - for % of upper threshold defining lower hysteris threshold
    cv.createTrackbar(trackbarUpperThresholdRatio_name, window_settings_name, 0,
                      trackbarUpperThresholdRatio_max_value, process_display_callback)

    # Set trackbar default positions
    # - for Gaussian kernel std deviation
    trackbarStdDeviation_value = (int)(sigma_default_value / trackbarStdDeviation_stepsize)
    cv.setTrackbarPos(trackbarStdDeviation_name, window_settings_name,
                      trackbarStdDeviation_value)
    
    # - for contrast norm
    if useL2NormForContrast_default_value == True:
        trackbarContrastNorm_value = 1
    else:
        trackbarContrastNorm_value = 0
    cv.setTrackbarPos(trackbarContrastNorm_name, window_settings_name,
                      trackbarContrastNorm_value)
    
    # - for upper hysteresis threshold
    trackbarUpperThreshold_value = (int)(upperThreshold_default_value)
    cv.setTrackbarPos(trackbarUpperThreshold_name, window_settings_name,
                      trackbarUpperThreshold_value)
    
    # - for % of upper threshold defining lower hysteris threshold
    trackbarUpperThresholdRatio_value = (int)(upperThresholdRatio_default_value);
    cv.setTrackbarPos(trackbarUpperThresholdRatio_name, window_settings_name,
                      trackbarUpperThresholdRatio_value)
    
#
# Input & Preprocessing
#
def load_preprocess(filepath):
    global image_in, the_image_in, image_overlay
    
    # Input
    # - Load image
    image_in = cv.imread(filepath, cv.IMREAD_UNCHANGED)
    
    #- Check if the image is valid
    if image_in is None:
        sys.exit("! Cannot read the image " + filepath)
    
    # Convert image to graylevel if appropriate
    if image_in.ndim == 2:
        the_image_in = image_in
    else:
        the_image_in = cv.cvtColor(image_in, cv.COLOR_BGR2GRAY)
        
    # Create edge map overlay BGR image
    if (image_in.ndim == 2):
        nrows, ncols = np.shape(image_in)
    else:
        nrows, ncols, nchannels = np.shape(image_in)
    image_overlay = np.empty((nrows, ncols, 3), dtype="uint8")

#
# Trackbar callback
#
def process_display_callback(value):
    global sigma                 # Gaussian std deviation
    global upperThreshold        # Upper threshold
    global upperThresholdRatio   # % of upper threshold, defining lower threshold
    global lowerThreshold        # Lower threshold
    global useL2NormForContrast  # Vector norm for contrast
                                 # L1-norm is used if "False"

    # Get trackbars positions
    trackbarStdDeviation_value = cv.getTrackbarPos(trackbarStdDeviation_name, window_settings_name)
    trackbarContrastNorm_value = cv.getTrackbarPos(trackbarContrastNorm_name, window_settings_name)
    trackbarUpperThreshold_value = cv.getTrackbarPos(trackbarUpperThreshold_name, window_settings_name)
    trackbarUpperThresholdRatio_value = cv.getTrackbarPos(trackbarUpperThresholdRatio_name, window_settings_name)
    
    # Set hyperparameter values from trackbars
    sigma = (float)(trackbarStdDeviation_value)*trackbarStdDeviation_stepsize
    if trackbarContrastNorm_value == 1:
        useL2NormForContrast = True
    else:
        useL2NormForContrast = False
    upperThreshold = (float)(trackbarUpperThreshold_value)
    upperThresholdRatio = (float)(trackbarUpperThresholdRatio_value)
    lowerThreshold = 0.01*upperThreshold*upperThresholdRatio
    
    if verbosity == True:
        if useL2NormForContrast == True:
            norm = "L2"
        else:
            norm = "L1"
        print("Std Dev =", sigma, "|", norm, "Norm", "| Upp =", upperThreshold, "| Low =", lowerThreshold)
    
    # Processing & Visualization
    process_display()

#
# Processing & Visualization
#
def process_display():
    global image_out, image_overlay
    
    # Preprocessing > Gaussian filtering
    # - Kernel size is set to 0, and is automatically estimated from sigma
    # - Last argument "BorderType" of GaussianBlur() is omitted,
    #   so that default boundary conditions (BORDER_DEFAULT) are used
    image_out = cv.GaussianBlur(the_image_in, (0,0), sigma)
  
    # Gradient-based edge detection using the pipeline:
    #   Sobel contrast map > Non-Maximum Suppression > Hysteresis Threshold
    image_out = cv.Canny(image_out, lowerThreshold, upperThreshold, image_out,
                         SobelKernelSize, useL2NormForContrast)

    # Display results
    # - Edge map
    cv.imshow(window_out_name, image_out)
    
    # - Edge map overlaid on original image
    image_overlay = overlay_uchar_image(the_image_in, image_out,
                                        edge_color, edge_color3, image_overlay)
    cv.imshow(window_overlay_name, image_overlay)

##
## MAIN ROUTINE
##

def application(filepath):
    # Input & Preprocessing
    load_preprocess(filepath)

    # GUI creation
    create_GUI()

    # Processing & Visualization
    # - Display original image 
    cv.imshow(window_in_name, the_image_in)

    # - Invoke callback routine to initialize and process
    process_display_callback(trackbarStdDeviation_value)

    # Event loop > Wait for pressed key
    cv.waitKey(0)
        
    # Destroy windows
    cv.destroyAllWindows()

<b>Run the application</b>

In [ ]:
# - Image directory
imagedir = "/content/drive/MyDrive/Images"

# - Input image
filename = "Tiger.jpg"

# - Silent / Verbose mode
verbosity = False

# Run application
application(imagedir+filename)